In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments/")
print(os.getcwd())

/content
/content/drive/My Drive/Uni Projects/Anomaly Detection using Ensembles/Experiments


In [ ]:
from models.models import MnistModel

In [ ]:
model0 = MnistModel()
classifier0 = model0.build()
classifier0 = model0.load_model("models/classifier0")

model1 = MnistModel()
classifier1 = model1.build()
classifier1 = model1.load_model("models/classifier1")

model2 = MnistModel()
classifier2 = model2.build()
classifier2 = model2.load_model("models/classifier2")

model3 = MnistModel()
classifier3 = model3.build()
classifier3 = model3.load_model("models/classifier3")

model4 = MnistModel()
classifier4 = model4.build()
classifier4 = model4.load_model("models/classifier4")

model5 = MnistModel()
classifier5 = model5.build()
classifier5 = model5.load_model("models/classifier5")

model6 = MnistModel()
classifier6 = model6.build()
classifier6 = model6.load_model("models/classifier6")

model7 = MnistModel()
classifier7 = model7.build()
classifier7 = model7.load_model("models/classifier7")

model8 = MnistModel()
classifier8 = model8.build()
classifier8 = model8.load_model("models/classifier8")

In [ ]:
pickle_in = open("pre-defined/mnist_data_wo9.pickle","rb")
x_train, y_train, x_val, y_val, x_test, y_test = pickle.load(pickle_in)

In [ ]:
pickle_in = open("pre-defined/temperature_value.pickle","rb")
temp_value = pickle.load(pickle_in)

In [ ]:
ind = 1
ood = 0

**Calculating new dataset which contains softmax distriution of each data**

In [ ]:
softmax_dataset = []
for img, lbl in tqdm(zip(x_val,y_val)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    label = ood
  else:
    label = ind 
  logits = model0.model(img)
  logits = tf.math.divide(logits, temp_value[0])
  pred = model0.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 1:
    label = ood
  else:
    label = ind 
  logits = model1.model(img)
  logits = tf.math.divide(logits, temp_value[1])
  pred = model1.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 2:
    label = ood
  else:
    label = ind 
  logits = model2.model(img)
  logits = tf.math.divide(logits, temp_value[2])
  pred = model2.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 3:
    label = ood
  else:
    label = ind 
  logits = model3.model(img)
  logits = tf.math.divide(logits, temp_value[3])
  pred = model3.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 4:
    label = ood
  else:
    label = ind 
  logits = model4.model(img)
  logits = tf.math.divide(logits, temp_value[4])
  pred = model4.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 5:
    label = ood
  else:
    label = ind 
  logits = model5.model(img)
  logits = tf.math.divide(logits, temp_value[5])
  pred = model5.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 6:
    label = ood
  else:
    label = ind 
  logits = model6.model(img)
  logits = tf.math.divide(logits, temp_value[6])
  pred = model6.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 7:
    label = ood
  else:
    label = ind 
  logits = model7.model(img)
  logits = tf.math.divide(logits, temp_value[7])
  pred = model7.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)

  if lbl == 8:
    label = ood
  else:
    label = ind 
  logits = model8.model(img)
  logits = tf.math.divide(logits, temp_value[8])
  pred = model8.apply_softmax(logits)
  data = []
  data = [pred,label]
  softmax_dataset.append(data)
 

10817it [02:40, 67.37it/s]


In [ ]:
softmax_dataset = np.array(softmax_dataset)

In [ ]:
softmax_dataset[0]

array([<tf.Tensor: shape=(1, 9), dtype=float32, numpy=
array([[4.4095266e-04, 9.9245775e-01, 7.0967095e-04, 5.7334756e-04,
        6.9927378e-04, 4.7506217e-04, 1.4127033e-03, 1.4520351e-04,
        3.0861411e-03]], dtype=float32)>,
       1], dtype=object)

In [ ]:
count = 0
for d in softmax_dataset:
  if d[1] == ind:
    count = count + 1

print(count)

86536


In [ ]:
pickle_out = open("pre-defined/method2_dataset.pickle","wb")
pickle.dump(softmax_dataset, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("pre-defined/method2_dataset.pickle","rb")
softmax_dataset = pickle.load(pickle_in)

**Since IN data will be much more it will cause class imbalance problem thats why only considering class distribution to be same for OOD and IN**

In [ ]:
ind_data = []
ood_data = []
for data in softmax_dataset:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
ind_data = ind_data[:len(ood_data)]

In [ ]:
len(ind_data)

10817

In [ ]:
input = []
labels = []
for idata, odata in zip(ind_data,ood_data):
  input.append(idata[0][0])
  input.append(odata[0][0])
  labels.append(idata[1])
  labels.append(odata[1])  

In [ ]:
input[0]

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([4.4095266e-04, 9.9245775e-01, 7.0967095e-04, 5.7334756e-04,
       6.9927378e-04, 4.7506217e-04, 1.4127033e-03, 1.4520351e-04,
       3.0861411e-03], dtype=float32)>

In [ ]:
xm2_train, xm2_test, ym2_train, ym2_test = train_test_split(input, labels, test_size=0.20)

In [ ]:
xm2_test

[<tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([1.5535867e-05, 9.6414267e-04, 4.6007225e-01, 5.1602781e-01,
        7.0305881e-03, 1.5878124e-04, 3.6626043e-06, 1.8037335e-03,
        1.3923483e-02], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([8.3266683e-03, 4.7078658e-05, 2.7444038e-01, 1.6514804e-03,
        1.4402359e-06, 4.8228988e-01, 5.1491760e-04, 1.2982667e-04,
        2.3259832e-01], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([1.3339691e-05, 7.1416578e-10, 7.4062759e-06, 6.0835422e-04,
        5.5772665e-07, 8.6309034e-01, 1.2208689e-04, 5.2847823e-05,
        1.3610508e-01], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([7.70962448e-04, 9.69862103e-01, 9.16393194e-03, 3.92429065e-03,
        1.68020884e-03, 2.41048168e-04, 1.32776354e-03, 8.21470283e-04,
        1.22082615e-02], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([2.0053309e-05, 3.6381778e-10, 2.3899824e

**Training SVM on this new dataset**

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(SVC(gamma='auto'))
clf.fit(xm2_train, ym2_train)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
ym2_pred = clf.predict(xm2_test)

In [ ]:
clf.score(xm2_test, ym2_test)

0.8479315923272475

In [ ]:
for t, p in zip(ym2_test, ym2_pred):
  print("Actual Label: ", t, "Predicted Label: ", p)

Actual Label:  0 Predicted Label:  0
Actual Label:  0 Predicted Label:  0
Actual Label:  0 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  0 Predicted Label:  0
Actual Label:  0 Predicted Label:  0
Actual Label:  0 Predicted Label:  0
Actual Label:  1 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  0 Predicted Label:  1
Actual Label:  0 Predicted Label:  0
Actual Label:  0 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  0 Predicted Label:  0
Actual Label:  1 Predicted Label:  1
Actual Label:  0 Predicted Label:  1
Actual Label:  0 Predicted Label:  0
Actual Label:  0 Predicted Label:  1
Actual Label:  1 Predicted Label:  1
Actual Label:  0 Predicted Label:  1
Actual Label:  0 Predicted Label:  1
Actual Label:  0 Predicted Label:  1
Actual Label:  0 Predicted Label:  0
Actual Label:  1 Predicted Label:  1
A

**Testing on the test data which has class 9. Class 9 will be considered anomaly and rest as IN data**

In [ ]:
accuracy = 0.0
acc_in = 0.0
acc_ood = 0.0
count_in = 0
count_ood = 0
for data, lbl in zip(x_test, y_test):
  output = []
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 9:
    label = ood
    count_ood = count_ood + 1
  else:
    label = ind
    count_in = count_in + 1

  logits = model0.model(img)
  logits = tf.math.divide(logits, temp_value[0])
  pred = model0.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model1.model(img)
  logits = tf.math.divide(logits, temp_value[1])
  pred = model1.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model2.model(img)
  logits = tf.math.divide(logits, temp_value[2])
  pred = model2.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model3.model(img)
  logits = tf.math.divide(logits, temp_value[3])
  pred = model3.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model4.model(img)
  logits = tf.math.divide(logits, temp_value[4])
  pred = model4.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model5.model(img)
  logits = tf.math.divide(logits, temp_value[5])
  pred = model5.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model6.model(img)
  logits = tf.math.divide(logits, temp_value[6])
  pred = model6.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model7.model(img)
  logits = tf.math.divide(logits, temp_value[7])
  pred = model7.apply_softmax(logits)
  output.append(clf.predict(pred))

  logits = model8.model(img)
  logits = tf.math.divide(logits, temp_value[2])
  pred = model8.apply_softmax(logits)
  output.append(clf.predict(pred))

  values, counts = np.unique(output, return_counts=True)
  y_pred = values[np.argmax(counts)]

  if y_pred == label:
    accuracy = accuracy + 1
    if label == ind:
      acc_in = acc_in + 1
    else:
      acc_ood = acc_ood + 1


print("Total Accuracy: ", accuracy/len(x_test)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)

Total Accuracy:  0.9375
Accuracy of determining ID data:  0.9476142809476142
Accuracy of determining OOD data:  0.8473736372646185
